<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/vertex_ai_test/vertex_ai_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
# ddfimport.ddf_source_control_pane()
ddfimport.ddf_import_common()


Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 7.36 KiB | 7.36 MiB/s, done.
Resolving deltas: 100% (7/7), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [4]:
import importlib
import eeddf
importlib.reload(eeddf)
import eeraster
importlib.reload(eeraster)

import numpy as np
import time
import ee

In [3]:
# This may prompt for your Google account credentials to authenticate for
# Earth Engine. You may pass in an argument test_environment = True to access
# test data.
eeddf.initialize_ddf(test_environment = True)

In [ ]:
  # "VPD" : vapor_pressure_deficit_geotiff,
  # "RH": relative_humidity_geotiff,
  # "PET": pet_geotiff,
  # "DEM": dem_geotiff,
  # "PA": pa_geotiff,
  # "Mean Annual Temperature": temperature_geotiff,
  # "Mean Annual Precipitation": brazil_map_geotiff,
  # "Iso_Oxi_Stack_mean_TERZER": craig_gordon_isoscape_geotiff,
  # "isoscape_fullmodel_d18O_prec_REGRESSION": precipitation_regression_isoscape_geotiff,
  # "brisoscape_mean_ISORIX": brisoscape_geotiff,
  # "d13C_cel_mean": d13C_mean_geotiff,
  # "d13C_cel_var": d13C_var_geotiff,
  # "ordinary_kriging_linear_d18O_predicted_mean" : krig_means_isoscape_geotiff,
  # "ordinary_kriging_linear_d18O_predicted_variance" : krig_variances_isoscape_geotiff

In [15]:
vpd = ee.Image('projects/river-sky-386919/assets/reference_rasters/vpd').select("b1").rename("VPD")
rh = ee.Image('projects/river-sky-386919/assets/reference_rasters/vpd').select("b1").rename("RH")
pet = ee.Image('projects/river-sky-386919/assets/reference_rasters/pet').select("b1").rename("PET")
dem = ee.ImageCollection('projects/sat-io/open-datasets/GLO-30').select("b1").mosaic().rename("DEM")
pa = ee.Image('projects/river-sky-386919/assets/reference_rasters/dem_pa_brasil_raster').select("b2").rename("PA")
temp = ee.Image('projects/river-sky-386919/assets/reference_rasters/mean_annual_temperature').select("b1").rename("Mean Annual Temperature")
prec = ee.Image('WORLDCLIM/V1/BIO').select('bio12').rename("Mean Annual Precipitation")
cg = ee.Image('projects/river-sky-386919/assets/reference_rasters/Iso_Oxi_Stack_mean_TERZER').select('b1').rename("Iso_Oxi_Stack_mean_TERZER")
prec_reg = ee.Image('projects/river-sky-386919/assets/reference_rasters/isoscape_fullmodel_d18O_prec_REGRESSION').select('b1').rename("isoscape_fullmodel_d18O_prec_REGRESSION")
brisoisorix = ee.Image('projects/river-sky-386919/assets/reference_rasters/brisoscape_mean_ISORIX').select('b1').rename("brisoscape_mean_ISORIX")
d13C_mean = ee.Image('projects/river-sky-386919/assets/reference_rasters/d13C_cel_map_BRAZIL_stack').select('b1').rename("d13C_cel_mean")
d13C_var = ee.Image('projects/river-sky-386919/assets/reference_rasters/d13C_cel_map_BRAZIL_stack').select('b2').rename("d13C_cel_var")
krig_mean = ee.Image('projects/river-sky-386919/assets/reference_rasters/uc_davis_d18O_cel_ordinary_random_grouped_means').select('b1').rename("ordinary_kriging_linear_d18O_predicted_mean")
krig_var = ee.Image('projects/river-sky-386919/assets/reference_rasters/uc_davis_d18O_cel_ordinary_random_grouped_vars').select('b1').rename("ordinary_kriging_linear_d18O_predicted_variance")

image_features = [vpd, rh, pet, dem, pa, temp, prec, cg, prec_reg, brisoisorix, d13C_mean, d13C_var, krig_mean, krig_var]

# Add all the other images as bands to the first image.
aggregated = image_features[0]
for image in range(1, len(image_features)):
  aggregated.addBands(image)

print(aggregated.getInfo())


{'type': 'Image', 'bands': [{'id': 'VPD', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [941, 937], 'crs': 'EPSG:4326', 'crs_transform': [0.04166666666665718, 0, -74.0000000000241, 0, -0.04166666666667143, 5.29166666665704]}], 'version': 1695924701571346, 'id': 'projects/river-sky-386919/assets/reference_rasters/vpd', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-66.63541667943487, -33.77090919107404], [-64.18749999991525, -33.77090919144714], [-59.29166664498704, -33.77090915005782], [-51.947916685933684, -33.77090917632496], [-47.05208330827284, -33.77090913361833], [-44.60416669353532, -33.770909152117], [-39.70833330612979, -33.77090916461001], [-34.7623011318398, -33.770403423678886], [-34.770591137859284, 5.3124066571087925], [-39.70833330612978, 5.312573202501962], [-49.49999997085615, 5.312573218424768], [-59.29166664498704, 5.312573178621252], [-69.08333337187567, 5.312573243178015], [-74.02107547204307, 5.312406629037761